In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
from scipy import stats
import os
import nibabel as nib
import nilearn
from nilearn import plotting, image
from nilearn import input_data
import matplotlib.pyplot as plt
from nilearn.input_data import NiftiLabelsMasker, NiftiMasker

In [3]:
Subject = "606"
Run = "2"
stim_site_coord = [(59, -56, 47)]
data_path = "/Users/hellwalker/Desktop/scans/" + Subject + "/Run" + Run + "/preproc_functinal.nii.gz"
thalamus_mask_path = "/Users/hellwalker/Desktop/scans/Morel_Mask_LPI_2mm.nii.gz"
ROI_mask_path = '/Users/hellwalker/Desktop/scans/Gordon_333_cortical_2mm.nii.gz'

## Utility Functions

In [4]:
def apply_mask(image, mask_image):
    masker = NiftiMasker(mask_img=mask_image)
    masker.fit()
    signal = masker.transform(image)
    return signal

In [5]:
def get_correlation_dict(time_series, seed):
    """timeseries # : correlation between that timeseries and the seed."""
    correlation_dict = {}
    for i, time_serie in enumerate(time_series):
        correlation_dict[i] = sp.stats.pearsonr(seed, time_serie)[0]
    return correlation_dict

In [6]:
def get_highest_correlation_key(correlation_dict):
    highest_correlation_key = max(correlation_dict, key=lambda x: correlation_dict[x])
    print("best is element", highest_correlation_key, "with correlation", correlation_dict[highest_correlation_key])
    return highest_correlation_key

## Loading the Image

In [7]:
"""Load image"""
img = nib.load(data_path)
img_data = img.get_data()

## Getting the Stimulation Site Time Series

In [8]:
stim_site_label = list(stim_site_coord)
stim_site_masker = input_data.NiftiSpheresMasker(seeds = stim_site_coord, radius = 4, allow_overlap = True)
stim_site_time_series = stim_site_masker.fit_transform(img)
stim_site_time_series = stim_site_time_series.reshape((600,))

## Finding the best correlated thalamus voxel with stim_site

In [9]:
"""Loading thalamus_mask"""
thalamus_mask_img = nib.load(stim_mask_path)
thalamus_mask_data = thalamus_mask_img.get_data()

NameError: name 'stim_mask_path' is not defined

In [ ]:
thalamus_time_series = apply_mask(img, thalamus_mask_img)
thalamus_corr_dict = get_correlation_dict(thalamus_time_series.T, stim_site_time_series)
best_thalamus_voxel = get_highest_correlation_key(thalamus_corr_dict)
best_thalamus_voxel_time_series = thalamus_time_series.T[best_thalamus_voxel]

## Computing the time series for 333 ROIs

In [ ]:
ROI_labels = np.arange(333)
#plotting.plot_roi(atlas_filename)
ROI_masker = NiftiLabelsMasker(labels_img=ROI_mask_path)
global ROI_time_series
ROI_time_series = ROI_masker.fit_transform(img)

In [ ]:
def get_top_20(ROI_time_series, seed_time_series):
    ROI_corr_dict = get_correlation_dict(ROI_time_series.T, seed_time_series)
    best_ROI = get_highest_correlation_key(ROI_corr_dict)
    import operator
    sorted_dict = sorted(ROI_corr_dict.items(), key=operator.itemgetter(1))
    best_20_with_corr = sorted_dict[-20:][::-1]
    best_20 = [item[0] for item in best_20_with_corr]
    return best_20

## Finding the best correlated 20 ROIs with stim_site and best_thalamus_voxel

In [ ]:
print("Subject", Subject, "Run", Run)
print("---")
stim_site_ROI_rank = get_top_20(ROI_time_series, stim_site_time_series) 
print("stim_site_ROI_rank:", stim_site_ROI_rank)
print("---")
best_thalamus_voxel_ROI_rank = get_top_20(ROI_time_series, best_thalamus_voxel_time_series) 
print("best_thalamus_voxel_ROI_rank:", best_thalamus_voxel_ROI_rank)
print("---")
print("common ROIs:", [i for i in stim_site_ROI_rank if i in best_thalamus_voxel_ROI_rank])
print("---")